In [89]:
import torch
device="cuda" if torch.cuda.is_available() else "cpu"

In [1]:
!pip install -q torch transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

dataset = load_dataset("squad")
print(dataset["train"][0])  # View first sample


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}


In [60]:
# Subsample the dataset
small_train_dataset = dataset["train"].select(range(80000))  # First 1000 samples
small_validation_dataset = dataset["validation"].select(range(8000))  # First 100 samples

# Update the dataset dictionary
small_dataset = {
    "train": small_train_dataset,
    "validation": small_validation_dataset,
}

In [61]:
from transformers import AutoTokenizer

# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


In [62]:

def preprocess(example):
    # Tokenize the question and context
    tokenized_input = tokenizer(
        example["question"],
        example["context"],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_offsets_mapping=True  # Required to map tokens back to character positions
    )

    # Get the character start and end positions of the answer
    answer_start = example["answers"]["answer_start"][0]
    answer_text = example["answers"]["text"][0]

    # Find the token positions corresponding to the answer
    start_char = answer_start
    end_char = answer_start + len(answer_text)

    # Convert character positions to token positions
    sequence_ids = tokenized_input.sequence_ids()
    offset_mapping = tokenized_input["offset_mapping"]

    # Find the start and end token positions
    start_token = None
    end_token = None
    for idx, (start, end) in enumerate(offset_mapping):
        if start <= start_char < end:
            start_token = idx
        if start < end_char <= end:
            end_token = idx
        if start_token is not None and end_token is not None:
            break

    # If the answer is not fully contained in the tokenized input, set start and end to 0
    if start_token is None or end_token is None:
        start_token = 0
        end_token = 0

    # Add start and end positions to the tokenized input
    tokenized_input["start_positions"] = start_token
    tokenized_input["end_positions"] = end_token

    return tokenized_input

In [63]:
# Preprocess the smaller dataset
small_tokenized_dataset = dataset.map(
    preprocess,
    batched=False
)

In [64]:
# Filter the tokenized dataset to the desired size
small_tokenized_dataset = {
    "train": small_tokenized_dataset["train"].select(range(1000)),
    "validation": small_tokenized_dataset["validation"].select(range(100)),
}
# C

In [65]:
# Convert to a DatasetDict object for easier manipulation
from datasets import DatasetDict
small_tokenized_dataset = DatasetDict(small_tokenized_dataset)

# Set format for PyTorch
small_tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "start_positions", "end_positions"])

In [95]:
class QA_Model(nn.Module):
    def __init__(self):
        super(QA_Model, self).__init__()
        self.encoder = AutoModel.from_pretrained("distilbert-base-uncased")
        self.dropout = nn.Dropout(0.1)  # Add dropout
        self.qa_head = nn.Linear(self.encoder.config.hidden_size, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids, attention_mask=attention_mask)
        outputs = self.dropout(outputs.last_hidden_state)  # Apply dropout
        logits = self.qa_head(outputs)
        start_logits, end_logits = logits.split(1, dim=-1)
        return start_logits.squeeze(-1), end_logits.squeeze(-1)

In [96]:
# Initialize the model and move it to GPU
model = QA_Model().to("cuda")



In [97]:
from torch.optim import AdamW
# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5,weight_decay=0.01)

# Define the loss function
loss_fn = nn.CrossEntropyLoss()

In [98]:
from torch.utils.data import DataLoader

# Create DataLoader for the smaller dataset
small_train_dataloader = DataLoader(small_tokenized_dataset["train"], batch_size=8, shuffle=True)
small_validation_dataloader = DataLoader(small_tokenized_dataset["validation"], batch_size=8)

In [99]:
def train_model(model, train_dataloader, validation_dataloader, optimizer, loss_fn, epochs, device="cuda"):
    patience = 3  # Number of epochs to wait for improvement
    best_validation_loss = float("inf")
    epochs_without_improvement = 0
    model.to(device)

    for epoch in range(epochs):
        # Training phase
        model.train()
        total_train_loss = 0

        for batch in train_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            start_positions = batch["start_positions"].to(device)
            end_positions = batch["end_positions"].to(device)

            # Forward pass
            start_logits, end_logits = model(input_ids, attention_mask)

            # Compute loss
            start_loss = loss_fn(start_logits, start_positions)
            end_loss = loss_fn(end_logits, end_positions)
            loss = start_loss + end_loss

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        # Validation phase
        validation_loss = evaluate_model(
            model=model,
            validation_dataloader=validation_dataloader,
            loss_fn=loss_fn,
            device=device
        )

        # Early stopping logic
        if validation_loss < best_validation_loss:
            best_validation_loss = validation_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            print(f"Early stopping at epoch {epoch + 1}")
            break

In [100]:
import torch
def evaluate_model(model, validation_dataloader, loss_fn, device="cuda"):
    """
    Evaluates the model on the validation dataset.

    Args:
        model: The model to evaluate.
        validation_dataloader: DataLoader for the validation dataset.
        loss_fn: Loss function.
        device: Device to use for evaluation (default: "cuda").

    Returns:
        The total validation loss.
    """
    model.eval()
    model.to(device)
    total_eval_loss = 0

    with torch.no_grad():
        for batch in validation_dataloader:
            # Move batch to the specified device
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            start_positions = batch["start_positions"].to(device)
            end_positions = batch["end_positions"].to(device)

            # Forward pass
            start_logits, end_logits = model(input_ids, attention_mask)

            # Compute loss
            start_loss = loss_fn(start_logits, start_positions)
            end_loss = loss_fn(end_logits, end_positions)
            loss = start_loss + end_loss

            total_eval_loss += loss.item()

    return total_eval_loss

In [101]:
# Define the number of epochs
epochs = 20

# Call the training function
train_model(
    model=model,
    train_dataloader=small_train_dataloader,
    validation_dataloader=small_validation_dataloader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=epochs,
    device="cuda"
)

Early stopping at epoch 6


In [102]:
def predict_answer(model, tokenizer, question, context, device="cuda"):
    """
    Predicts an answer to a given question based on the provided context.

    Args:
        model: The trained QA model.
        tokenizer: The tokenizer used for preprocessing.
        question: The question to answer.
        context: The context in which to find the answer.
        device: Device to use for prediction (default: "cuda").

    Returns:
        The predicted answer.
    """
    # Tokenize the input
    inputs = tokenizer(
        question,
        context,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )

    # Move inputs to the specified device
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Set the model to evaluation mode
    model.eval()

    # Perform inference
    with torch.no_grad():
        start_logits, end_logits = model(input_ids, attention_mask)

    # Get the predicted start and end positions
    start_pred = torch.argmax(start_logits, dim=-1).item()
    end_pred = torch.argmax(end_logits, dim=-1).item()

    # Decode the predicted answer
    predicted_answer = tokenizer.decode(
        input_ids[0][start_pred:end_pred + 1],
        skip_special_tokens=True
    )

    return predicted_answer

In [103]:
# Define a custom question and context
question = "Why was the Eiffel Tower initially criticized?"
context = "The Eiffel Tower, located in Paris, France, was completed in 1889. It was designed by Gustave Eiffel and was initially criticized by some of France's leading artists and intellectuals. However, it has become a global cultural icon and one of the most recognizable structures in the world."

# Call the prediction function
predicted_answer = predict_answer(
    model=model,
    tokenizer=tokenizer,
    question=question,
    context=context,
    device="cuda"
)

print(f"Question: {question}")
print(f"Predicted Answer: {predicted_answer}")

Question: Why was the Eiffel Tower initially criticized?
Predicted Answer: some of france ' s leading artists and intellectuals
